In [ ]:
!pip install awpy
!pip install openpyxl

In [21]:
import os
import pandas as pd
from awpy import Demo
import time

In [22]:
def round_score_df(round):
    df = round.pivot(index='round_num', columns='winner',values='winner').reset_index()
    df['ct'] = df['ct'].notna().astype(int)
    df['t'] = df['t'].notna().astype(int)
    # Cumulative score
    df['ct_score'] = df['ct'].cumsum()
    df['t_score'] = df['t'].cumsum()

    #current score
    df['ct_current'] = df['ct_score'].shift(fill_value=0)
    df['t_current'] = df['t_score'].shift(fill_value=0)

    # Format as "ct-t"
    df['score'] = df['ct_current'].astype(str) + "-" + df['t_current'].astype(str)
    round_score = df
    
    return round_score

In [23]:
def kills_df(kills):
    kills_df = kills[['round_num','attacker_name','weapon']]
    my_kills = kills_df[kills_df['attacker_name']=="FX"].groupby(['round_num']).count().reset_index()
    my_kills = my_kills[['round_num','attacker_name']]
    
    return my_kills

In [24]:
def my_highlights(rounds,kills):
    my_highlights = pd.merge(kills,rounds,on='round_num',how='left')[['round_num','attacker_name','score']]
    my_highlights.rename(columns={'attacker_name':'FX_kills'})

    return my_highlights

In [44]:

file_path = r'D:\CS2 Demos'

workbook_path = r'D:\\my_highlights.xlsx'

if os.path.isfile(workbook_path):
    my_highlights_df = pd.read_excel(workbook_path)
    prsnt_dm_file = my_highlights_df['file_name'].unique().tolist()
else:
    my_highlights_df = pd.DataFrame(columns=['round_num','FX_kills','score','time','map','file_name'])

list_dem = os.listdir(file_path)

prsnt_dm_file = [os.path.join(file_path,s) for s in prsnt_dm_file]

#removing the files that has .info at end
demos = [s for s in list_dem if '.info' not in s]
demos = [os.path.join(file_path,s) for s in demos if os.path.join(file_path,s) not in prsnt_dm_file]

for demo in demos:

    dem = Demo(file_path_demo)
    dem.parse()

    cs_map = dem.header['map_name']


    rnds = dem.rounds.to_pandas()
    klls = dem.kills.to_pandas()

    rnds_df = round_score_df(rnds)
    klls_df = kills_df(klls)

    highlight = my_highlights(rnds_df,klls_df)

    file_stats = os.stat(file_path_demo)

    highlight['time'] = time.ctime(file_stats.st_ctime)
    highlight['map'] = cs_map
    highlight['file_name'] = file_path_demo

    my_highlights_df = pd.concat([my_highlights_df,highlight])
    print(f'Done Parsing....{file_path_demo}')

my_highlights_df.to_excel(workbook_path, index=False)
print('Done!')


Done Parsing....D:\CS2 Demos\003776257228009373930_0950681461.dem
Done!
